## Iterators and Generators

Python iterators are objects with the `__next__` and `__iter__` methods defined. Other objects in the iterator eco-system are:

- `next(iterator[, default])` Retrieve the next item from the iterator by calling its `__next__()` method. If `default` is given, it is returned if the iterator is exhausted, otherwise `StopIteration` is raised.

-  `iter(object[, sentinel])` Return an iterator object. The first argument is interpreted very differently depending on the presence of the second argument. Without a second argument, object must be a collection object which supports the iteration protocol (the `__iter__()` method), or it must support the sequence protocol (the `__getitem__()` method with integer arguments starting at `0`). If it does not support either of those protocols, `TypeError` is raised. If the second argument, sentinel, is given, then object must be a callable object. The iterator created in this case will call object with no arguments for each call to its `__next__()` method; if the value returned is equal to sentinel, `StopIteration` will be raised, otherwise the value will be returned.

- `StopIteration` Raised by built-in function `next()` and an iterator’s `__next__()` method to signal that there are no further items produced by the iterator. If a generator code directly or indirectly raises `StopIteration`, it is converted into a `RuntimeError` (retaining the `StopIteration` as the new exception’s cause).

In [1]:
class Integers:
    """All positive integers."""
    def __iter__(self):
        self.x = 0
        return self
        
    def __next__(self):
        self.x += 1
        return self.x

In [2]:
from itertools import islice

for i in islice(Integers(), 5):
    print(i, end=' ')

1 2 3 4 5 

### Exercise: even integers

> <div class="alert alert-block alert-info">
> Write an iterator iterating all even positive integers.
> </div>

## Generators

In [3]:
def integers():
    x = 1
    while True:
        yield x
        x += 1

In [4]:
list(islice(integers(), 20))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

A *generator* is like a function that remembers where it was, the last time it generated a value.

In [5]:
def is_prime(x):
    return x >= 2 and all(x % i != 0 for i in range(2, x//2 + 1))

> <div class="alert alert-info">
>    Do you understand how the <tt>is_prime</tt> function works?
>    <tt>range</tt> Gives an iterator, do you see another iterator being used in the above definition?
> </div>

In [6]:
[i for i in range(50) if is_prime(i)]

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]

The above list-comprehension performs a filtration. Another way to write this:

In [7]:
list(filter(is_prime, range(50)))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]

### Exercise: cumsum

> <div class="alert alert-block alert-info">
>     Write an iterator that generates the cumulative sum of the primes.
> </div>

In [8]:
def cumsum(lst):
    total = 0
    for x in lst:
        total += x
        yield total
    return

In [9]:
from itertools import count

primes = filter(is_prime, count(1))
prime_sum = cumsum(primes)

In [10]:
list(islice(prime_sum, 10))  # run this several times

[2, 5, 10, 17, 28, 41, 58, 77, 100, 129]

In [11]:
# fix:
def prime_sum():
    primes = filter(is_prime, count(1))
    return cumsum(primes)

In [12]:
list(islice(prime_sum(), 10))  # run this several times

[2, 5, 10, 17, 28, 41, 58, 77, 100, 129]

### Exercise: intersperse

> <div class="alert alert-info">
> Write a generator that intersperses two sequences:
>
>     >>> intersperse([1, 2, 3], ["one", "two", "three"])
>     [1, "one", 2, "two", 3, "three"]
> </div>

In [13]:
def intersperse(a, b):
    a = iter(a)
    b = iter(b)
    try:
        while True:
            yield next(a)
            yield next(b)
    except StopIteration:
        return

In [14]:
list(intersperse([1, 2, 3], ["one", "two", "three"]))

[1, 'one', 2, 'two', 3, 'three']

In [15]:
list(intersperse(count(1), ["one", "two", "three"]))

[1, 'one', 2, 'two', 3, 'three', 4]

### Context manager

You may use a generator to create a context manager.

# Co-routines

The generator pattern plays with the stack based model of function evaluation. When Python evaluates an expression like

```py
(1 + 2) * (3 + 4)
```

what happens internally is:

> create a stack frame:
>
> > compute 1 + 2
> >
> > return 3
>
> save 3
>
> create a stack frame:
>
> > compute 3 + 4
> >
> > return 7
>
> save 7
>
> compute 3 * 7
> 
> return 21

A stack-frame is usually associated with a localized environment with its own space of variables. After the computation has finished, the stack-frame (and its associated environment) is garbage collocted.

In the case of a co-routine the environment is saved and can be reused later in the computation. The co-routine can be seen as a little independent thread of computation (microthread, greenlet, torque, stackless, twisted, etc.). In low level Python this is just a slight change from calling an ordinary function. This is why generators can be relatively efficient.

Currently our understanding of co-routines allows for one-way extraction of values in the generator style. Python also allows us to put values into a co-routine.

In [51]:
def printer():
    line = 0
    while True:
        x = yield
        print("{:04X}:".format(line), x)
        line += 1

In [52]:
p = printer()
next(p)

In [54]:
for x in range(16):
    p.send(x)

0010: 0
0011: 1
0012: 2
0013: 3
0014: 4
0015: 5
0016: 6
0017: 7
0018: 8
0019: 9
001A: 10
001B: 11
001C: 12
001D: 13
001E: 14
001F: 15


## AsyncIO

This way of defining co-routines is now depricated in favour of `async`/`await`.

In [ ]:
async def printer():
    line = 0
    while True:
        x = await